### Проверка того, что наше api работает

Перед началом необходимо собрать docker-образ и запустить контейнер

1. Следовать инструкциям https://github.com/fimochka-sudo/GB_docker_flask_example/blob/main/README.md
2. Далее уже запускать код ниже

In [1]:
import pandas as pd
from sklearn.metrics import roc_auc_score,roc_curve
from urllib import request, parse

X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [2]:
X_test[['age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo',
        'cholesterol', 'gluc', 'smoke', 'alco', 'active']].head(3)

,age,height,weight,ap_hi,ap_lo,gender,cholesterol,gluc,smoke,alco,active
0,17561,162,75.0,120,80,2,2,2,1,0,1
1,22098,164,65.0,120,80,1,1,1,0,0,1
2,15921,154,61.0,100,60,1,1,1,0,0,0


In [3]:
import urllib.request
import json      


def get_prediction(x):
    description, company_profile, benefits = x
    body = {'age': age,
            'gender': gender,
            'height': height,
            'weight': weight,
            'ap_hi': ap_hi,
            'ap_lo': ap_lo,            
            'cholesterol': cholesterol,
            'gluc': gluc,
            'smoke': smoke,
            'alco': alco,
            'active': active} 

    myurl = "http://0.0.0.0:8180/predict"
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    #print (jsondataasbytes)
    response = urllib.request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['predictions']

In [12]:
#get_prediction(1,10,1,1)

In [4]:
%%time
predictions = X_test[['age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo',
                      'cholesterol', 'gluc', 'smoke', 'alco', 'active'
                     ]].iloc[:500].apply(lambda x: get_prediction(x), 1)

CPU times: user 511 ms, sys: 75.8 ms, total: 587 ms
Wall time: 6.71 s


In [6]:
predictions

0      0.026174
1      0.044576
2      0.004804
3      0.001057
4      0.001229
         ...   
495    0.008572
496    0.001839
497    0.620032
498    0.001418
499    0.001099
Length: 500, dtype: float64

In [5]:
roc_auc_score(y_score=predictions.values, y_true=y_test.iloc[:500])

0.9803161348046525

In [7]:
#X_test[['description', 'company_profile', 'benefits']].head(3)

Давайте подумаем, где наше api может сломаться

In [10]:
get_prediction(("", None, None))

0.13518841848029955

In [9]:
get_prediction(("", "", ""))

0.13518841848029955